## counting inversion


In [1]:
def Count_Inversion(arr : list):
    if(len(arr) <= 1):
        return arr, 0
    
    start = 0
    end = len(arr)
    mid = (start + end) // 2
    arr1, inv1 = Count_Inversion(arr[start:mid])
    arr2, inv2 = Count_Inversion(arr[mid:end])
    i, j, merge_inv = 0, 0, 0
    curr_arr = []
    while (i < len(arr1) and j < len(arr2)):
        if(arr1[i] <= arr2[j]):
            curr_arr.append(arr1[i])
            i += 1
        else:
            merge_inv += len(arr1) - i
            curr_arr.append(arr2[j])
            j += 1
    
    #copy remaining array
    if(i == len(arr1)):
        while j < len(arr2):
            curr_arr.append(arr2[j])
            j += 1
    else:
        while i < len(arr1):
            curr_arr.append(arr1[i])
            i += 1


    return curr_arr, (inv1 + inv2 + merge_inv)
    


## check if solvable

In [2]:
def is_solvable(grid : list[list[int]]) -> bool:
    k = len(grid)
    flattened_grid = [item for arr in grid for item in arr]
    blank_index = flattened_grid.index(0)
    flattened_grid.pop(blank_index)
    _, inversions = Count_Inversion(flattened_grid)
    print("number of inversions : ", inversions)
    
    # k odd
    if(k % 2):
        return not(inversions % 2)
    else:
        blank_index_from_bottom = k - (blank_index // 2)
        return bool((blank_index_from_bottom % 2) ^ (inversions % 2))


## Different heuristics

hamming distance

In [3]:
def hamming_distance(grid : list[list[int]]) -> int:
    flattened_grid = [item for arr in grid for item in arr]
    dist = 0
    for i, num in enumerate(flattened_grid):
        if(num != 0 and i + 1 != num):
            dist += 1
    
    return dist

Manhattan Distance

In [4]:
def manhattan_distance(grid : list[list[int]]) -> int:
    dist = 0
    k = len(grid)
    for row, arr in enumerate(grid):
        for col, num in enumerate(arr):
            if(num == 0):
                continue
            target_row = (num - 1) // k
            target_col = (num - 1) % k
            curr_dist = abs(row - target_row) + abs(col - target_col)
            dist += curr_dist
            #print(f"manhattan dist for {num} = {curr_dist}")
        
    return dist

Euclidean Distance

In [5]:
import math
def euclidean_distance(grid : list[list[int]]) -> int:
    dist = 0
    k = len(grid)
    for row, arr in enumerate(grid):
        for col, num in enumerate(arr):
            if(num == 0):
                continue
            target_row = (num - 1) // k
            target_col = (num - 1) % k
            curr_dist = (abs(row - target_row))**2 + (abs(col - target_col))**2
            dist += math.sqrt(curr_dist)
            #print(f"euclidean dist squared for {num} = {curr_dist}")

    return dist.__round__(3)

Linear Conflict


In [6]:
def row_conflicts(arr, row, num, num_index):
    conflicts = 0
    k = len(arr)
    for i,elem in enumerate(arr):
        target_row = (elem - 1) // k
        if(row == target_row and num > elem and num_index < i):
            conflicts += 1
            #print(f"conflict in row {row} bw {num} and {elem}")
    
    return conflicts

def col_conflicts(arr, col, num, num_index):
    conflicts = 0
    k = len(arr)
    for i, elem in enumerate(arr):
        target_col = (elem - 1) % k
        if(col == target_col and num > elem and num_index < i):
            conflicts += 1
            #print(f"conflict in col {col} bw {num} and {elem}")

    return conflicts

def linear_conflict(grid : list[list[int]]) -> int:
    conflicts = 0
    k = len(grid)
    #row wise
    for row, arr in enumerate(grid):
        for col, num in enumerate(arr):
            target_row = (num - 1) // k
            target_col = (num - 1) % k
            if(target_row == row):
                conflicts += row_conflicts(arr, row, num, col)
            if(target_col == col):
                col_arr = [row[col] for row in grid]
                conflicts += col_conflicts(col_arr, col, num, row)
    
    #print(f"no of linear conflicts : {conflicts}")
    return manhattan_distance(grid) + (2 * conflicts)
            
            


## create puzzle class

In [7]:
from typing import Callable
import copy

class N_Puzzle:
    def __init__(self, initial_grid : list[list[int]], parent : 'N_Puzzle' = None):
        self.grid = initial_grid
        self.moves_count = 0
        self.parent = parent
        self.k = len(initial_grid)
        self.priority = 0

    def get_blank_index(self):
        for i, row in enumerate(self.grid):
            for j, val in enumerate(row):
                if val == 0:
                   return i,j

    def generate_config(self):
        row_blank, col_blank = self.get_blank_index()
        valid_configs = []
        #up move allowed
        if(row_blank > 0):
            new_grid_up = copy.deepcopy(self.grid)
            target_num = new_grid_up[row_blank-1][col_blank]
            new_grid_up[row_blank][col_blank] = target_num
            new_grid_up[row_blank-1][col_blank] = 0
            valid_configs.append(new_grid_up)

        #down move allowed
        if(row_blank < (self.k - 1)):
            new_grid_down = copy.deepcopy(self.grid)
            target_num = new_grid_down[row_blank + 1][col_blank]
            new_grid_down[row_blank][col_blank] = target_num
            new_grid_down[row_blank + 1][col_blank] = 0
            valid_configs.append(new_grid_down)
            

        #left move allowed
        if(col_blank > 0):
            new_grid_left = copy.deepcopy(self.grid)
            target_num = new_grid_left[row_blank][col_blank - 1]
            new_grid_left[row_blank][col_blank] = target_num
            new_grid_left[row_blank][col_blank - 1] = 0
            valid_configs.append(new_grid_left)
            
        #right move allowed
        if(col_blank < (self.k - 1)):
            new_grid_right = copy.deepcopy(self.grid)
            target_num = new_grid_right[row_blank][col_blank + 1]
            new_grid_right[row_blank][col_blank] = target_num
            new_grid_right[row_blank][col_blank + 1] = 0
            valid_configs.append(new_grid_right)
        
        return valid_configs
    
    def is_correct_config(self) -> bool:
        flattened_grid = [item for row in self.grid for item in row]
        for i, val in enumerate(flattened_grid):
            if(i + 1 != val and val != 0):
                return False
        
        return True
    
    def is_solvable(self) -> bool:
        
        k = len(self.grid)
        flattened_grid = [item for arr in self.grid for item in arr]
        blank_index = flattened_grid.index(0)
        flattened_grid.pop(blank_index)
        _, inversions = Count_Inversion(flattened_grid)
        #print("number of inversions : ", inversions)
        
        # k odd
        if(k % 2):
            return not(inversions % 2)
        else:
            blank_index_from_bottom = k - (blank_index // 2)
            return bool((blank_index_from_bottom % 2) ^ (inversions % 2))

    
    def __str__(self):
        string = ''
        for rows in self.grid:    
            for elem in rows:
                if(elem == 0):
                    string += "- "
                    continue
                string += str(elem) + " "
            string += '\n' 
        
        return string
            


## create a priority queue

In [8]:
import heapq
import itertools

class PriorityQueue:
    def __init__(self):
        self.heap = []
        self.counter = itertools.count()  #to distinguish if priority same

    def push(self, priority : int, item : 'N_Puzzle'):
        count = next(self.counter)
        # Add a tuple: (priority, count, item)
        heapq.heappush(self.heap, (priority, count, item))

    def pop(self) -> tuple[int, 'N_Puzzle']:
        if self.heap:
            priority, count, item = heapq.heappop(self.heap)
            return priority, item
        else:
            raise IndexError("pop from an empty priority queue")
    
    def size(self):
        return len(self.heap)

        

## Take input

In [9]:
k = int(input("enter matrix size then the elements"))
n = k*k
print(f"{n}, {k}")

matrix = []
for i in range(k):
    input_str = input("")
    arr = list(map(int, input_str.split(" ")))
    print(f'string is = {input_str} and corresponding array is {arr}')
    matrix.append(arr)


9, 3
string is = 1 5 0 and corresponding array is [1, 5, 0]
string is = 7 6 4 and corresponding array is [7, 6, 4]
string is = 2 3 8 and corresponding array is [2, 3, 8]


## create inital config

In [11]:
initial_state = N_Puzzle(matrix)
if not(initial_state.is_solvable()):
    print("Unsolvable puzzle")

else:
    pqueue = PriorityQueue()
    pqueue.push(0, initial_state)

    heuristics = [hamming_distance, manhattan_distance, euclidean_distance, linear_conflict]
    h_n = heuristics[1]

    # open_list : list[N_Puzzle] = []
    #closed_list : list[N_Puzzle] = []
    closed_set: set = set()  # To store string representations of grids
    i = 0
    explored = 0
    expanded = 0
    while pqueue.size() != 0:
        i += 1
        print(f"iteration number : {i}")
        _, curr_state = pqueue.pop()
        #closed_list.append(curr_state)
        closed_set.add(str(curr_state.grid))
        expanded += 1
        #print(curr_state)
        if(curr_state.is_correct_config()):
            break
        
        possible_configs = curr_state.generate_config()
        for config in possible_configs:
            nei_puzzle = N_Puzzle(config)
            nei_puzzle.moves_count = curr_state.moves_count + 1
            nei_puzzle.priority = nei_puzzle.moves_count + h_n(nei_puzzle.grid)
            nei_puzzle.parent = curr_state
            if str(nei_puzzle.grid) not in closed_set:
                pqueue.push(nei_puzzle.priority, nei_puzzle)
                explored += 1


    print(f"SOLVED PUZZLE in {curr_state.moves_count} moves")



iteration number : 1
iteration number : 2
iteration number : 3
iteration number : 4
iteration number : 5
iteration number : 6
iteration number : 7
iteration number : 8
iteration number : 9
iteration number : 10
iteration number : 11
iteration number : 12
iteration number : 13
iteration number : 14
iteration number : 15
iteration number : 16
iteration number : 17
iteration number : 18
iteration number : 19
iteration number : 20
iteration number : 21
iteration number : 22
iteration number : 23
iteration number : 24
iteration number : 25
iteration number : 26
iteration number : 27
iteration number : 28
iteration number : 29
iteration number : 30
iteration number : 31
iteration number : 32
iteration number : 33
iteration number : 34
iteration number : 35
iteration number : 36
iteration number : 37
iteration number : 38
iteration number : 39
iteration number : 40
iteration number : 41
iteration number : 42
iteration number : 43
iteration number : 44
iteration number : 45
iteration number : 

In [12]:
states_path = []
while curr_state.parent is not None:
    states_path.append(curr_state)
    curr_state = curr_state.parent

states_path.append(curr_state)
states_path = states_path[::-1]

In [14]:
print("Closed List : ", expanded)
print("open list : ", explored)

Closed List :  1137
open list :  1802


In [15]:
for states in states_path:
    print(states)

1 5 - 
7 6 4 
2 3 8 

1 5 4 
7 6 - 
2 3 8 

1 5 4 
7 - 6 
2 3 8 

1 5 4 
7 3 6 
2 - 8 

1 5 4 
7 3 6 
2 8 - 

1 5 4 
7 3 - 
2 8 6 

1 5 - 
7 3 4 
2 8 6 

1 - 5 
7 3 4 
2 8 6 

1 3 5 
7 - 4 
2 8 6 

1 3 5 
7 4 - 
2 8 6 

1 3 5 
7 4 6 
2 8 - 

1 3 5 
7 4 6 
2 - 8 

1 3 5 
7 4 6 
- 2 8 

1 3 5 
- 4 6 
7 2 8 

1 3 5 
4 - 6 
7 2 8 

1 3 5 
4 2 6 
7 - 8 

1 3 5 
4 2 6 
7 8 - 

1 3 5 
4 2 - 
7 8 6 

1 3 - 
4 2 5 
7 8 6 

1 - 3 
4 2 5 
7 8 6 

1 2 3 
4 - 5 
7 8 6 

1 2 3 
4 5 - 
7 8 6 

1 2 3 
4 5 6 
7 8 - 

